# Signals Boosting

NOTE: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [ ]:
%run ./1.setting-up-the-retrotech-dataset.ipynb

## Keyword Search with No Signals Boosting

### Listing 4.5

In [ ]:
query = "ipad"
collection = "products"

engine = get_engine()
response = engine.search(collection, products_request(query))
display(HTML(render_search_results(query, engine.docs_as_html(response))))

## Create Signals Boosts (Signals Aggregation)

### Listing 4.6

In [ ]:
products_collection="products"
signals_collection="signals"
signals_boosting_collection="signals_boosting"

create_collection(signals_boosting_collection)

print("Aggregating Signals to Create Signals Boosts...")
signals_opts={"zkhost": "aips-zk", "collection": signals_collection}
df = spark.read.format("solr").options(**signals_opts).load()
df.createOrReplaceTempView("signals")

signals_aggregation_query = """
select q.target as query, c.target as doc, count(c.target) as boost
  from signals c left join signals q on c.query_id = q.query_id
  where c.type = 'click' AND q.type = 'query'
  group by query, doc
  order by boost desc
"""

signals_boosting_opts={"zkhost": "aips-zk", "collection": signals_boosting_collection, 
                       "gen_uniq_key": "true", "commit_within": "5000"}
spark.sql(signals_aggregation_query).write.format("solr").options(**signals_boosting_opts).mode("overwrite").save()
print("Signals Aggregation Completed!")

## Search with Signals Boosts Applied

### Listing 4.7

In [ ]:
query = "ipad"
collection = "products"
signals_boosts_query = {
    "query": query,
    "fields": ["doc", "boost"],
    "limit": 10,
    "params": {
      "defType": "edismax",
      "qf": "query",
      "sort": "boost desc"
    }
}
engine = get_engine()
response = engine.search(collection, signals_boosts_query)
print(f"Boost Documents: \n{response}")

product_boosts = " ".join(
    [f'"{entry["doc"]}"^{str(entry["boost"])}' 
     for entry in response])
print(f"\nBoost Query: \n{product_boosts}")

collection = "products"
request = products_request()
request["params"]["boost"] = "sum(1,query({! df=upc v=$signals_boosting}))"
request["params"]["signals_boosting"] = product_boosts
response = engine.search(collection, request)
display(HTML(render_search_results(query, response)))

## Success!

You have now implemented your first AI-powered search algorithm: Signals Boosting. This is an overly simplistic implementation (we'll dive much deeper into signals boosting improvements in chapter 8), but it demonstrates the power of leveraging reflected intelligence quite well. We will dive into other Reflected Intelligence techniques in future chapters, such as Collaborative Filtering (in chapter 9 - Personalized Search) and Machine-learned Ranking (in chapter 10 - Learning to Rank).

Up next: Chapter 5 - [Knowledge Graph Learning](../ch05/1.open-information-extraction.ipynb)

